In [1]:
# Import necessary libraries
import pandas as pd
import parse

In [30]:
# Define paths to data
train = "data/sled_categorization_TRAINSMALL_PREPROCESSED.txt"

test = "data/sled_categorization_TEST_PREPROCESSED.txt"

dev = "data/sled_categorization_DEV_PREPROCESSED.txt"

## Analyse data

In [31]:
# Parse the file to create a dataframe from them
def parse_file(path: str):
    """Reads fasttext formatted file and returns labels, texts."""
    with open(path, "r") as f:
        content = f.readlines()
    pattern = "__label__{label} {text}\n"
    p = parse.compile(pattern)

    labels, texts = list(), list()
    for line in content:
        rez = p.parse(line)
        if rez is not None:
            labels.append(rez["label"])
            texts.append(rez["text"])
        else:
            print("error parsing line ", line)
    dataframe = pd.DataFrame({"label": labels, "text": texts})
    return dataframe

In [32]:
train_df = parse_file(train)
train_df

,label,text
0,crnakronika,na tolminskem se je dopoldne zgodila prometna ...
1,crnakronika,na cesti zadlaz-žabče se je zgodila prometna n...
2,crnakronika,v sredo ob 1321 je bila novogoriška policija o...
3,crnakronika,malo po 16 uri je v svetem duhu na ostrem vrhu...
4,crnakronika,v eksploziji pirotehnike je bil v soboto popol...
...,...,...
9998,znanost,12 julij na ta dan bodo vesoljske agencije obj...
9999,znanost,kitajska je do zdaj izvedla pet uspešnih odpra...
10000,znanost,da sta mikro- in nanoplastika povsod okoli nas...
10001,znanost,znanstveni dosežek je podprla tudi whitney cas...


In [33]:
train_df["split"] = "train"

train_df.head()

,label,text,split
0,crnakronika,na tolminskem se je dopoldne zgodila prometna ...,train
1,crnakronika,na cesti zadlaz-žabče se je zgodila prometna n...,train
2,crnakronika,v sredo ob 1321 je bila novogoriška policija o...,train
3,crnakronika,malo po 16 uri je v svetem duhu na ostrem vrhu...,train
4,crnakronika,v eksploziji pirotehnike je bil v soboto popol...,train


In [34]:
test_df = parse_file(test)
test_df["split"] = "test"
dev_df = parse_file(dev)
dev_df["split"] = "dev"

In [35]:
test_df.shape

(1300, 3)

In [36]:
# Merge the dataframes
temp = pd.concat((train_df, test_df))
temp.shape

(11303, 3)

In [37]:
df = pd.concat((temp, dev_df))
df.shape

(12603, 3)

In [38]:
df.describe(include="all")

,label,text,split
count,12603,12603,12603
unique,13,12585,3
top,crnakronika,v silovitem neurju ki je v soboto prizadelo fr...,train
freq,1000,2,10003


In [39]:
# Check whether there are any duplicates
df[df.text.duplicated(keep=False)].sort_values("text")

,label,text,split
5590,prosticas,berem da je prizemljil letalo da so ga ponekod...,train
5592,prosticas,berem da je prizemljil letalo da so ga ponekod...,train
5716,prosticas,glavna novost prenovljenega arteona je karavan...,train
5713,prosticas,glavna novost prenovljenega arteona je karavan...,train
8986,zdravje,lani je v slovenskih porodnišnicah rodilo okol...,train
8985,zdravje,lani je v slovenskih porodnišnicah rodilo okol...,train
9933,znanost,minister mark boris andrijanič je obisk začel ...,train
9932,znanost,minister mark boris andrijanič je obisk začel ...,train
5593,prosticas,na dan ko je vlada odločila da naj bodo vsi do...,train
5586,prosticas,na dan ko je vlada odločila da naj bodo vsi do...,train


In [40]:
# Drop duplicates
df.drop_duplicates("text",inplace=True)
df.describe()

,label,text,split
count,12585,12585,12585
unique,13,12585,3
top,crnakronika,na tolminskem se je dopoldne zgodila prometna ...,train
freq,1000,1,9990


In [42]:
# Analyze no. of texts in each split
print(df.split.value_counts().to_markdown())

|       |   split |
|:------|--------:|
| train |    9990 |
| test  |    1299 |
| dev   |    1296 |


In [18]:
print(list(df.label.unique()))

['crnakronika', 'druzba', 'gospodarstvo', 'izobrazevanje', 'kultura', 'okolje', 'politika', 'prosticas', 'sport', 'vreme', 'zabava', 'zdravje', 'znanost']


In [43]:
# Analyze label distribution
print(df.label.value_counts().to_markdown())

|               |   label |
|:--------------|--------:|
| crnakronika   |    1000 |
| druzba        |    1000 |
| gospodarstvo  |    1000 |
| politika      |    1000 |
| sport         |    1000 |
| zabava        |    1000 |
| zdravje       |     999 |
| kultura       |     998 |
| znanost       |     951 |
| vreme         |     926 |
| prosticas     |     912 |
| okolje        |     904 |
| izobrazevanje |     895 |


In [45]:
df.label.value_counts(normalize="True")

crnakronika      0.079460
druzba           0.079460
gospodarstvo     0.079460
politika         0.079460
sport            0.079460
zabava           0.079460
zdravje          0.079380
kultura          0.079301
znanost          0.075566
vreme            0.073580
prosticas        0.072467
okolje           0.071832
izobrazevanje    0.071116
Name: label, dtype: float64

In [44]:
# Let's analyze the text length

# Calculate text length
df["length"] = df["text"].str.split().str.len()

df.head(3)

,label,text,split,length
0,crnakronika,na tolminskem se je dopoldne zgodila prometna ...,train,152
1,crnakronika,na cesti zadlaz-žabče se je zgodila prometna n...,train,175
2,crnakronika,v sredo ob 1321 je bila novogoriška policija o...,train,127


In [46]:
# Analyze text length
print(df.length.describe().to_markdown())

|       |    length |
|:------|----------:|
| count | 12585     |
| mean  |   242.994 |
| std   |   109.83  |
| min   |    97     |
| 25%   |   143     |
| 50%   |   217     |
| 75%   |   313     |
| max   |  1382     |


## Prepare the data for Transformers

In [47]:
df.head()

,label,text,split,length
0,crnakronika,na tolminskem se je dopoldne zgodila prometna ...,train,152
1,crnakronika,na cesti zadlaz-žabče se je zgodila prometna n...,train,175
2,crnakronika,v sredo ob 1321 je bila novogoriška policija o...,train,127
3,crnakronika,malo po 16 uri je v svetem duhu na ostrem vrhu...,train,127
4,crnakronika,v eksploziji pirotehnike je bil v soboto popol...,train,116


In [49]:
# Create a df for the simple transformers - the first column should be "text" and the second "labels"
def prepare_data(dataframe, split):
    """ Creates a DataFrame for Simple Transformers and saves it. 
        Args:
        - dataframe: the initial dataframe
        - split: "train", "dev", or "test"    
    """
    prepared_df = dataframe[dataframe["split"] == split]
    final_df = pd.DataFrame({"text": prepared_df["text"],"labels":prepared_df["label"]})

    # Save the df
    final_df.to_csv(f"SLED-for-Transformers-{split}.csv", sep="\t")

    # Print information about the dataframe
    print(final_df.shape)

    return final_df

In [50]:
prepared_train = prepare_data(df, "train")

(9990, 2)


In [51]:
prepared_train.head(3)

,text,labels
0,na tolminskem se je dopoldne zgodila prometna ...,crnakronika
1,na cesti zadlaz-žabče se je zgodila prometna n...,crnakronika
2,v sredo ob 1321 je bila novogoriška policija o...,crnakronika


In [52]:
prepared_test = prepare_data(df, "test")

prepared_test.head()

(1299, 2)


,text,labels
0,v občini komen se je ponoči zgodila huda prome...,crnakronika
1,pobudnik spornega početja naj bi bil 20-letni ...,crnakronika
2,v bližini italijanske gorice le streljaj od ne...,crnakronika
3,56-letni voznik tovornega vozila je v novi gor...,crnakronika
4,na zatožno klop so sedli nekdanji dekan ekonom...,crnakronika


In [53]:
prepared_dev = prepare_data(df, "dev")

prepared_dev.head()

(1296, 2)


,text,labels
0,novomeški policisti so med kontrolo prometa na...,crnakronika
1,41-letni mariborčan je v ponedeljek okoli 22 u...,crnakronika
2,na slovenskih cestah sta se zgodili dve hudi p...,crnakronika
3,kriminalisti so na podlagi prijave dopps-a na ...,crnakronika
4,zoper šest osumljencev ki jim očitajo zlorabo ...,crnakronika
